# Bruker Behavior File Name Conversions Notebook
### Written by: Jeremy Delahanty, June 2021
----
As of 6.15.21, data that comes off the Bruker Microscope for behavior data is not organized or clean. Prairie View creates files that are either useless or not needed and, to clean up the directories, I wrote this notebook which will go through a project's directory and remove any unnecessary files.

This will be converted to a command line utility and, preferrably, something that will simply poll the server every now and again and perform cleanup whenever necessary for any and all projects collecting data off the Bruker microscope. Data should be named correctly without the user having to hand-hold files through this pipeline.


In [13]:
# Import pathlib for file naming and directory manipulation
import pathlib
import re
from pathlib import Path

In [7]:
# Give the noteboook a basepath to your data that is mapped to the SNL servers
# TODO: Automatically poll networked drives to see which one is nadata.snl.salk.edu\snlktdata
# TODO: Make this an input where the user can do this by just saying which project they want to look at
basepath = Path("Y:/specialk/chronic_mild_stress/") # Change to your project folder here...

#### Testing path
# basepath = "C:/Users/jdelahanty.SNL/Desktop/testdir/behavior"

In [8]:
# Bruker creates "Reference" tiff images that appear to be useless. To clean the directory, these are deleted.

# First, use Path.glob() for these files. Path.glob() gives a generator that we can iterate through.
reference_tiff_removal = Path(basepath).glob("*\References\*.tif")

# For each tiff in the files generated by Path.glob()
for tiff in reference_tiff_removal:
    print(tiff)
    
    # Delete the tiff file using unlink() and print what was removed
    tiff.unlink()
    print(tiff, "removed")

# You can only remove folders once the directory is empty. The above statement accomplishes this, so now create a generator for the directories themselves to iterate through.
reference_dir_removal = Path(basepath).glob("*\References")

# For each directory generated by the reference_dir_removal
for directory in reference_dir_removal:
    
    # Delete the directory using rmdir() and print what was removed
    directory.rmdir()
    print(directory, "removed")
    

Y:\specialk\chronic_mild_stress\20210706_CSC001_training1_plane0-085\References\20210706_CSC001_training1_plane0-085-Ch2-16bit-Reference.tif
Y:\specialk\chronic_mild_stress\20210706_CSC001_training1_plane0-085\References\20210706_CSC001_training1_plane0-085-Ch2-16bit-Reference.tif removed
Y:\specialk\chronic_mild_stress\20210706_CSC001_training1_plane0-085\References\20210706_CSC001_training1_plane0-085-Window1-Ch2-8bit-Reference.tif
Y:\specialk\chronic_mild_stress\20210706_CSC001_training1_plane0-085\References\20210706_CSC001_training1_plane0-085-Window1-Ch2-8bit-Reference.tif removed
Y:\specialk\chronic_mild_stress\20210706_CSC002_training1_plane0-086\References\20210706_CSC002_training1_plane0-086-Ch2-16bit-Reference.tif
Y:\specialk\chronic_mild_stress\20210706_CSC002_training1_plane0-086\References\20210706_CSC002_training1_plane0-086-Ch2-16bit-Reference.tif removed
Y:\specialk\chronic_mild_stress\20210706_CSC002_training1_plane0-086\References\20210706_CSC002_training1_plane0-086

In [20]:
# We'll now generate "raw" directories for each session behavior directories

# Use glob() to make a session generator
session_generator = Path(basepath).glob("*")

# Create string for raw directory creation
rawpath = "raw"

# For each session in the generator
for session in session_generator:
    
    # Rename the session to include the rawpath; the "/" operator appends the string rawpath to the Path generated
    session = session / rawpath
    
    # If session/raw is a directory, do nothing. Just print that the directory already exists
    if session.is_dir():
        print(session, "already exists!")
    
    # If it doesn't exist, make it.
    else:
        print("Making directory", session)
        Path(session).mkdir()

# Having gone through the whole generator using the above for loop, we have to reinitialize it
session_generator = Path(basepath).glob("*")

# Now that the sessions all include raw data directories, we need to move the raw files into the new folder

# For each session from the generator
for session in session_generator:
    
    # Determine if the content of "behavior" is a file or a directory
    # If it's a file, use list-comprehension to make a list of files in the directory
    files = [item for item in session.iterdir() if item.is_file()]
    # If it's a directory, use list-comprehension to make a list of that directory
    # TODO: See if using list comprehension is necessary for this...
    raw_dir = [directory for directory in session.iterdir() if directory.is_dir()]
    
    # Now that we have a list of files, we need to move them using Path.rename()
    # For each file in the file list
    for file in files:
        
        # Rename the file with a new path. Creates a path that uses "session/raw/filename" and then tells user it's been done.
        file.rename(Path.joinpath(raw_dir[0], file.name))
        print("moved", file.name)

Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210602_LHE011_plane0-034\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210602_LHE012_plane0-036\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210602_LHE013_plane0-037\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210602_LHE014_plane0-038\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210602_LHE015_plane0-039\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210602_LHE016_plane0-040\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210603_LHE011_plane0-042\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210603_LHE012_plane0-043\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210603_LHE013_plane0-044\raw
Making directory C:\Users\jdelahanty.SNL\Desktop\testdir\behavior\20210603_LHE014_plane0-045\raw
Making directory C:\Users\jdel

In [38]:
my_path = "Y:/specialk/chronic_mild_stress/CSE006/twop"

date_gen = Path(my_path).glob("*")

date_regex = re.compile("\d{8}")

date_folders = []

for i in date_gen:
    if i.is_dir():
        search = re.search(pattern=date_regex, string=i.name)
        date_folders.append(search.group(0))
        
for date in date_folders:
    new_path = Path(my_path) / date
    print(new_path)
    Path.mkdir(new_path)

Y:\specialk\chronic_mild_stress\CSE006\twop\20210706


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Y:\\specialk\\chronic_mild_stress\\CSE006\\twop\\20210706'

In [24]:


for date in date_folders:
    new_path = mypath
    Path.mkdir()

['20210706', '20210707', '20210708', '20210709', '20210710', '20210716', '20210723']
